<a href="https://colab.research.google.com/github/Ehsan0071/Ehsan0071-Final-Project-Semester-C-image-captioning/blob/Ehsan0071/Project_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing & Loading Modules

In [2]:
!pip install datasets -q
!pip install transformers -q
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import os
import subprocess
import datasets

def download_dataset():
    # Check if the dataset directory already exists
    if not os.path.exists('data'):
        os.makedirs('data')
        print("Directory 'data' created")
        # If you have a shell script for downloading, uncomment the next line
        # subprocess.call(['./download.sh'])
        print("Dataset downloaded.")
    else:
        print("Dataset directory already exists.")

def load_dataset():
    # Load the dataset from the local directory
    ds = datasets.load_dataset("atasoglu/flickr8k-dataset", data_dir="data")
    print("Dataset loaded successfully.")
    return ds

if __name__ == "__main__":
    # Download the dataset (if not already downloaded)
    download_dataset()
    # Load the dataset
    dataset = load_dataset()
    # Print the dataset structure
    print(dataset)



ModuleNotFoundError: No module named 'datasets'